# Preparação de Dados para Dashboard – Obesidade

Este notebook tem como objetivo preparar uma versão “analítica” da base `Obesity.csv` para uso em um painel
no Looker Studio. As transformações aqui realizadas incluem:
- padronização e renomeação de colunas para facilitar leitura e interpretação;
- criação de variáveis derivadas (ex.: target binário e IMC);
- arredondamento de variáveis ordinais com ruído decimal;
- exportação final do dataset para `data/processed/obesity_dashboard.csv`.

## Importações e configurações

In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

## Carregamento dos dados

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/RickPardono/TechChallenge4_Fiap/refs/heads/main/data/raw/Obesity.csv")
df.head()

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Obesity
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


## Checagens iniciais e estrutura

In [ ]:
df.shape

(2111, 17)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          2111 non-null   object 
 1   Age             2111 non-null   float64
 2   Height          2111 non-null   float64
 3   Weight          2111 non-null   float64
 4   family_history  2111 non-null   object 
 5   FAVC            2111 non-null   object 
 6   FCVC            2111 non-null   float64
 7   NCP             2111 non-null   float64
 8   CAEC            2111 non-null   object 
 9   SMOKE           2111 non-null   object 
 10  CH2O            2111 non-null   float64
 11  SCC             2111 non-null   object 
 12  FAF             2111 non-null   float64
 13  TUE             2111 non-null   float64
 14  CALC            2111 non-null   object 
 15  MTRANS          2111 non-null   object 
 16  Obesity         2111 non-null   object 
dtypes: float64(8), object(9)
memory u

In [ ]:
df.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


## Tratamento de variáveis ordinais com ruído decimal

Algumas variáveis ordinais (ex.: FCVC, NCP, CH2O, FAF, TUE) aparecem com valores decimais no arquivo,
mas representam categorias discretas. Para fins analíticos no dashboard, os valores serão arredondados para o
inteiro mais próximo.

In [ ]:
ordinal_noise = ["FCVC", "NCP", "CH2O", "FAF", "TUE"]

df[ordinal_noise] = df[ordinal_noise].apply(lambda col: np.rint(col).astype(int))

In [ ]:
df.head()

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Obesity
0,Female,21.0,1.62,64.0,yes,no,2,3,Sometimes,no,2,no,0,1,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3,3,Sometimes,yes,3,yes,3,0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2,3,Sometimes,no,2,no,2,1,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3,3,Sometimes,no,2,no,2,0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2,1,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation,Overweight_Level_II


## Criação de variáveis derivadas

Nesta etapa serão criadas variáveis que facilitam a análise no dashboard:
- 'ObeseBinary': target binário (obeso vs não obeso), alinhado ao problema do Tech Challenge;
- 'IMC': Índice de Massa Corporal (IMC), calculado a partir de peso e altura.

In [ ]:
obese_classes = {"Obesity_Type_I", "Obesity_Type_II", "Obesity_Type_III"}

df["ObeseBinary"] = df["Obesity"].apply(lambda x: 1 if x in obese_classes else 0)
df["IMC"] = (df["Weight"] / (df["Height"]**2)).round(2)

df[["Obesity", "ObeseBinary", "IMC"]].head()

,Obesity,ObeseBinary,IMC
0,Normal_Weight,0,24.39
1,Normal_Weight,0,24.24
2,Normal_Weight,0,23.77
3,Overweight_Level_I,0,26.85
4,Overweight_Level_II,0,28.34


## Criar rótulos amigáveis para o dashboard

Para facilitar a leitura no Looker Studio, algumas variáveis serão transformadas em rótulos interpretáveis.
Exemplos: 'ObeseBinary' → "Sim/Não", categorias ordinais → nomes descritivos.

In [ ]:
df["Obeso"] = df["ObeseBinary"].map({0: "Não", 1: "Sim"})

# Labels para ordinais (opcional, ajuda MUITO no Looker)
fcvc_map = {1: "Raramente", 2: "Às vezes", 3: "Sempre"}
ch2o_map = {1: "< 1L/dia", 2: "1–2L/dia", 3: "> 2L/dia"}
tue_map  = {0: "0–2h/dia", 1: "3–5h/dia", 2: "> 5h/dia"}
faf_map  = {0: "Nenhuma", 1: "1–2x/sem", 2: "3–4x/sem", 3: "5x+/sem"}
ncp_map  = {1: "1 refeição", 2: "2 refeições", 3: "3 refeições", 4: "4+ refeições"}

df["FCVC_label"] = df["FCVC"].map(fcvc_map)
df["CH2O_label"] = df["CH2O"].map(ch2o_map)
df["TUE_label"]  = df["TUE"].map(tue_map)
df["FAF_label"]  = df["FAF"].map(faf_map)
df["NCP_label"]  = df["NCP"].map(ncp_map)

df[["FCVC", "FCVC_label", "CH2O", "CH2O_label", "TUE", "TUE_label", "FAF", "FAF_label","NCP", "NCP_label"]].head()

,FCVC,FCVC_label,CH2O,CH2O_label,TUE,TUE_label,FAF,FAF_label,NCP,NCP_label
0,2,Às vezes,2,1–2L/dia,1,3–5h/dia,0,Nenhuma,3,3 refeições
1,3,Sempre,3,> 2L/dia,0,0–2h/dia,3,5x+/sem,3,3 refeições
2,2,Às vezes,2,1–2L/dia,1,3–5h/dia,2,3–4x/sem,3,3 refeições
3,3,Sempre,2,1–2L/dia,0,0–2h/dia,2,3–4x/sem,3,3 refeições
4,2,Às vezes,2,1–2L/dia,0,0–2h/dia,0,Nenhuma,1,1 refeição


## Renomeação de colunas

Nesta etapa as colunas são renomeadas para padronização e clareza, mantendo consistência com o notebook de EDA.
Isso facilitará a interpretação dos gráficos e métricas no painel.

In [ ]:
rename_map = {
    "Gender": "genero",
    "Age": "idade",
    "Height": "altura",
    "Weight": "peso",
    "family_history": "historico_familiar",
    "FAVC": "come_alimentos_caloricos",
    "FCVC": "consumo_vegetais",
    "NCP": "numero_refeicoes_diarias",
    "CAEC": "come_entre_refeicoes",
    "SMOKE": "fuma",
    "CH2O": "consumo_diario_agua",
    "SCC": "monitora_calorias",
    "FAF": "frequencia_atividade_fisica",
    "TUE": "tempo_dispositivos",
    "CALC": "consome_alcool",
    "MTRANS": "meio_transporte",
    "Obesity": "nivel_obesidade",
    "ObeseBinary": "obeso_binario",
    "IMC": "imc",
    "Obeso": "obeso",
    "FCVC_label": "consumo_vegetais_label",
    "CH2O_label": "consumo_agua_label",
    "TUE_label": "tempo_dispositivos_label",
    "FAF_label": "atividade_fisica_label",
    "NCP_label": "refeicoes_label",
}

df_dash = df.rename(columns=rename_map).copy()
df_dash.head()


,genero,idade,altura,peso,historico_familiar,come_alimentos_caloricos,consumo_vegetais,numero_refeicoes_diarias,come_entre_refeicoes,fuma,consumo_diario_agua,monitora_calorias,frequencia_atividade_fisica,tempo_dispositivos,consome_alcool,meio_transporte,nivel_obesidade,obeso_binario,imc,obeso,consumo_vegetais_label,consumo_agua_label,tempo_dispositivos_label,atividade_fisica_label,refeicoes_label
0,Female,21.0,1.62,64.0,yes,no,2,3,Sometimes,no,2,no,0,1,no,Public_Transportation,Normal_Weight,0,24.39,Não,Às vezes,1–2L/dia,3–5h/dia,Nenhuma,3 refeições
1,Female,21.0,1.52,56.0,yes,no,3,3,Sometimes,yes,3,yes,3,0,Sometimes,Public_Transportation,Normal_Weight,0,24.24,Não,Sempre,> 2L/dia,0–2h/dia,5x+/sem,3 refeições
2,Male,23.0,1.80,77.0,yes,no,2,3,Sometimes,no,2,no,2,1,Frequently,Public_Transportation,Normal_Weight,0,23.77,Não,Às vezes,1–2L/dia,3–5h/dia,3–4x/sem,3 refeições
3,Male,27.0,1.80,87.0,no,no,3,3,Sometimes,no,2,no,2,0,Frequently,Walking,Overweight_Level_I,0,26.85,Não,Sempre,1–2L/dia,0–2h/dia,3–4x/sem,3 refeições
4,Male,22.0,1.78,89.8,no,no,2,1,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation,Overweight_Level_II,0,28.34,Não,Às vezes,1–2L/dia,0–2h/dia,Nenhuma,1 refeição


## Seleção final de colunas para o dashboard

Aqui selecionamos as colunas que farão parte do dataset final do dashboard,
incluindo variáveis originais, variáveis derivadas e rótulos amigáveis.

In [ ]:
cols_dashboard = [
    "genero", "idade", "altura", "peso",
    "historico_familiar", "come_alimentos_caloricos", "fuma", "monitora_calorias",
    "consumo_vegetais", "numero_refeicoes_diarias", "come_entre_refeicoes",
    "consumo_diario_agua", "frequencia_atividade_fisica", "tempo_dispositivos",
    "consome_alcool", "meio_transporte",
    "nivel_obesidade", "obeso_binario", "obeso", "imc",
    "consumo_vegetais_label", "refeicoes_label", "consumo_agua_label",
    "atividade_fisica_label", "tempo_dispositivos_label"
]

df_dash_final = df_dash[cols_dashboard].copy()
df_dash_final.head()

,genero,idade,altura,peso,historico_familiar,come_alimentos_caloricos,fuma,monitora_calorias,consumo_vegetais,numero_refeicoes_diarias,come_entre_refeicoes,consumo_diario_agua,frequencia_atividade_fisica,tempo_dispositivos,consome_alcool,meio_transporte,nivel_obesidade,obeso_binario,obeso,imc,consumo_vegetais_label,refeicoes_label,consumo_agua_label,atividade_fisica_label,tempo_dispositivos_label
0,Female,21.0,1.62,64.0,yes,no,no,no,2,3,Sometimes,2,0,1,no,Public_Transportation,Normal_Weight,0,Não,24.39,Às vezes,3 refeições,1–2L/dia,Nenhuma,3–5h/dia
1,Female,21.0,1.52,56.0,yes,no,yes,yes,3,3,Sometimes,3,3,0,Sometimes,Public_Transportation,Normal_Weight,0,Não,24.24,Sempre,3 refeições,> 2L/dia,5x+/sem,0–2h/dia
2,Male,23.0,1.80,77.0,yes,no,no,no,2,3,Sometimes,2,2,1,Frequently,Public_Transportation,Normal_Weight,0,Não,23.77,Às vezes,3 refeições,1–2L/dia,3–4x/sem,3–5h/dia
3,Male,27.0,1.80,87.0,no,no,no,no,3,3,Sometimes,2,2,0,Frequently,Walking,Overweight_Level_I,0,Não,26.85,Sempre,3 refeições,1–2L/dia,3–4x/sem,0–2h/dia
4,Male,22.0,1.78,89.8,no,no,no,no,2,1,Sometimes,2,0,0,Sometimes,Public_Transportation,Overweight_Level_II,0,Não,28.34,Às vezes,1 refeição,1–2L/dia,Nenhuma,0–2h/dia


## Exportação final do dataset

O arquivo `obesity_dashboard.csv` será gerado a partir do DataFrame final preparado neste notebook.
Esse arquivo será utilizado como fonte de dados para a construção do painel analítico no Looker Studio.

In [ ]:
df_dash_final.to_csv("obesity_dashboard.csv", index=False)